# Displaying tub data as graphics
Enter the path to your car directory and the tub you want to plot from.

In [1]:
path = '/Users/dirk/mycar'
tub_id = 36

This find the tub through pattern matching and returns the first one, if there are multiple.

In [2]:
import glob
tub_path = glob.glob(path + '/data/tub_' + str(tub_id) + '_*')[0]
print(tub_path)

/Users/dirk/mycar/data/tub_36_20-03-13


This loads the tub through donkey.

In [3]:
import donkeycar as dk
from donkeycar.parts.datastore import Tub
cfg = dk.load_config(path + '/config.py')
tub = Tub(tub_path)

using donkey v3.1.1 ...
loading config file: /Users/dirk/mycar/config.py
loading personal config over-rides... 
config loaded
Tub exists: /Users/dirk/mycar/data/tub_36_20-03-13


Inspect the tub dimension (rows = num records, columns = data entries in record_xxx.json file):

In [4]:
import pandas as pd
df = tub.get_df()
print('tub shape:', df.shape)
print('columns:', df.columns.values)

tub shape: (12610, 10)
columns: ['cam/image_array' 'user/angle' 'user/throttle' 'car/speed'
 'car/inst_speed' 'car/lap' 'car/accel' 'car/gyro' 'timestamp'
 'milliseconds']


Use plotly to graph the values interactively.

In [5]:
dfa = df['car/accel'].apply(pd.Series)
dfa = dfa.rename(columns = lambda x : 'accel_' + str(x))
dfx = pd.concat([df[:], dfa], axis=1)
dfx['angle_scaled'] = dfx['user/angle'] * 10.0
dfx['speed_scaled'] = dfx['car/speed'] * 3
print(dfx)

                                         cam/image_array  user/angle  \
473    /Users/dirk/mycar/data/tub_36_20-03-13/473_cam...       0.150   
474    /Users/dirk/mycar/data/tub_36_20-03-13/474_cam...       0.150   
475    /Users/dirk/mycar/data/tub_36_20-03-13/475_cam...       0.150   
476    /Users/dirk/mycar/data/tub_36_20-03-13/476_cam...       0.152   
477    /Users/dirk/mycar/data/tub_36_20-03-13/477_cam...       0.142   
...                                                  ...         ...   
13150  /Users/dirk/mycar/data/tub_36_20-03-13/13150_c...       0.000   
13151  /Users/dirk/mycar/data/tub_36_20-03-13/13151_c...       0.000   
13152  /Users/dirk/mycar/data/tub_36_20-03-13/13152_c...       0.000   
13153  /Users/dirk/mycar/data/tub_36_20-03-13/13153_c...       0.000   
13154  /Users/dirk/mycar/data/tub_36_20-03-13/13154_c...       0.000   

       user/throttle  car/speed  car/inst_speed  car/lap  \
473            0.110  20.971780        0.524294        0   
474            

Here we plot the user angle vs the gyro x-axis.

In [8]:
l = ['angle_scaled', 'accel_0']

In [9]:
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()
t = dfx['timestamp']
for i in l:
    y = dfx[i]
    fig.add_trace(go.Scatter(x=t, y=y, name=i))
fig.update_layout(title_text='Donkey record of tub ' + str(tub_id),
                  xaxis_rangeslider_visible=True)
fig.show()

Plot data per lap:

In [12]:
import plotly.express as px
fig = px.line(df, y="user/angle", color="car/lap")
fig.show()